# Template: Using Helper Functions in Your Model Notebook

This notebook demonstrates how to integrate the helper utilities into your existing model evaluation notebooks.
It shows a minimal example using the new `dataset_utils`, `model_utils`, and `analysis_utils` modules.

In [ ]:
# SETUP: Import the helper utilities
import sys
from pathlib import Path

# Add src to path
src_path = Path("../src").resolve()
sys.path.insert(0, str(src_path))

# Import utilities
from dataset_utils import make_cityscapes_dataframe
from model_utils import BaseSegmentationModel, run_inference_over_df, evaluate_model_on_split
from analysis_utils import compute_image_statistics, identify_easy_vs_hard

print("✓ Helpers imported successfully")

In [ ]:
# STEP 1: Set up your model
# Your model must inherit from BaseSegmentationModel and implement predict()

from abc import abstractmethod
import torch
import numpy as np
from PIL import Image

class YourModelWrapper(BaseSegmentationModel):
    """Example wrapper for your model."""
    
    def __init__(self, model_name: str = "your-model-name"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Loading {model_name} on {self.device}...")
        # Load your model here
        # self.model = ... 
        
    def predict(self, image: Image.Image) -> np.ndarray:
        """
        Run inference and return segmentation mask.
        
        Args:
            image: PIL Image in RGB mode
            
        Returns:
            np.ndarray of shape (H, W) with integer trainIds (0-18)
        """
        # Implement your inference logic here
        # return pred_mask  # Should be np.ndarray with trainIds
        pass

# Instantiate
model = YourModelWrapper()

In [ ]:
# STEP 2: Set up paths and load dataset
CITYSCAPES_ROOT = Path("data/cityscapes")  # Update path as needed

# Load validation split into a DataFrame
val_df = make_cityscapes_dataframe(CITYSCAPES_ROOT, split="val")
print(f"Loaded {len(val_df)} validation images")

In [ ]:
# STEP 3: Run inference (once per model)
# Save predictions as .npy files

model_name = "YourModelWrapper"
pred_dir = CITYSCAPES_ROOT / f"{model_name}_preds" / "val"

# This runs inference and saves predictions
val_df_with_preds = run_inference_over_df(
    val_df,
    model=model,
    pred_root=pred_dir,
    overwrite=False  # Skip if already exists
)

print("✓ Inference complete")
print(val_df_with_preds[['image_id', 'pred_trainIds_path']].head())

In [ ]:
# STEP 4: Evaluate and save results
from model_utils import evaluate_model_on_split, save_results_csv

# Set up ground truth and results directories
gt_dir = CITYSCAPES_ROOT / "gtFine_trainvaltest" / "gtFine" / "val"
results_dir = CITYSCAPES_ROOT / "benchmark_results"

# Evaluate (computes IoU for each image/class)
results_df = evaluate_model_on_split(
    pred_dir=pred_dir,
    gt_dir=gt_dir,
    split_df=val_df,
    model_name=model_name,
)

# Save results
save_results_csv(results_df, results_dir, model_name)

print("✓ Evaluation complete")
print(f"Results shape: {results_df.shape}")
display(results_df.head())

In [ ]:
# STEP 5: Analyze results with helper functions
import matplotlib.pyplot as plt

# Compute image statistics
stats = compute_image_statistics(results_df)
print(f"Computed stats for {len(stats)} images")

# Identify easy and hard images
easy, hard = identify_easy_vs_hard(stats, n_images=10)

print(f"\nTop 5 EASIEST images:")
display(easy.head(5))

print(f"\nTop 5 HARDEST images:")
display(hard.head(5))

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(stats['mean_performance'], stats['moe_gain'], alpha=0.5)
ax.set_xlabel('Mean Model Performance (mIoU)')
ax.set_ylabel('MoE Potential Gain (max - mean)')
ax.set_title('Image Difficulty vs MoE Opportunity')
ax.grid(True, alpha=0.3)
plt.show()

## Quick Integration Tips

1. **Inherit from `BaseSegmentationModel`**: Your model wrapper should implement the `predict(image: Image.Image) -> np.ndarray` method.

2. **Use `run_inference_over_df()`**: Saves predictions to `.npy` files efficiently. Set `overwrite=False` to skip re-running.

3. **Use `evaluate_model_on_split()`**: Computes per-image, per-class IoU automatically. Results are saved as CSV.

4. **Use `compute_image_statistics()`**: Quickly identify easy vs hard images and MoE potential.

5. **For Hard Subset Analysis**: Import `create_hard_subset()` from `dataset_utils` and use `compare_subsets()` from `dataset_utils` to evaluate on the hard subset separately.

See `All_vs_Hard_Cityscapes.ipynb` for a complete example of hard subset evaluation.